In [1]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
print(mlflow.search_experiments()[0].name)

Default


### Q1

In [2]:
!mlflow --version

mlflow, version 2.3.2


### Q2

In [3]:
!python utils/preprocess_data.py --raw_data_path nyc_taxi_data --dest_path ./output

In [4]:
import os
print(f"DictVectorizer size in kb: {os.path.getsize('./output/dv.pkl') / 1024}")

DictVectorizer size in kb: 150.05859375


### Q3

In [5]:
!python utils/train.py

2023/06/04 22:30:33 INFO mlflow.tracking.fluent: Experiment with name 'HW2-nyc-taxi-experiment-training' does not exist. Creating a new experiment.


In [6]:
mlflow.search_experiments()[0].name

'HW2-nyc-taxi-experiment-training'

### Q4

In [7]:
!python utils/hpo.py

2023/06/04 22:31:27 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
[I 2023-06-04 22:31:27,351] A new study created in memory with name: no-name-2b4197f4-b2d7-4532-9259-00eff03816ff
[I 2023-06-04 22:31:30,976] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-04 22:31:31,394] Trial 1 finished with value: 2.4667366020368333 and parameters: {'n_estimators': 16, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.451379690825458.
[I 2023-06-04 22:31:34,419] Trial 2 finished with value: 2.449827329704216 and parameters: {'n_estimators': 34, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 2 with value: 2.449827329704216.
[I 2023-06-04 22:31:35,490] Trial 3 finished with value: 2.460983516558473 a

In [8]:
experiment_name = mlflow.search_experiments()[0].name
print(f"experiment name: {experiment_name}")
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
print(f"experiment id: {experiment_id}")

experiment name: random-forest-hyperopt
experiment id: 2


In [9]:
best_rmse = mlflow.search_runs(experiment_ids=experiment_id, order_by=["metrics.rmse ASC"]).loc[0, "metrics.rmse"]
print(f"Best RMSE: {best_rmse}")

Best RMSE: 2.449827329704216


### Q5

In [10]:
!python utils/register_model.py

Best model rmse: 2.2854691906481364


2023/06/04 22:31:45 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
Successfully registered model 'random-forest-model-tuned'.
2023/06/04 22:32:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-model-tuned, version 1
Created version '1' of model 'random-forest-model-tuned'.


### Q6

In [11]:
experiment_name = mlflow.search_experiments()[0].name
experiment_name

'random-forest-best-models'

In [12]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
client = MlflowClient("http://127.0.0.1:5000")

In [24]:
best_model = client.search_registered_models()[0]
best_model.name

'random-forest-model-tuned'

In [25]:
best_model.latest_versions[0].version

'1'

In [26]:
best_model.latest_versions[0].source

'mlflow-artifacts:/3/260e328c00f943eeb003fa84f5c9954d/artifacts/model'

In [27]:
model = mlflow.pyfunc.load_model(f"runs:/{best_run.info.run_id}/model")
model._model_meta

2023/06/04 22:39:30 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [28]:
model._model_meta._signature

inputs: 
  [Tensor('float64', (-1, 6683))]
outputs: 
  [Tensor('float64', (-1,))]